# Physical Models of Musical Instruments

* Pt 1: Guitar
* Pt 2: Piano
* Pt 3: Woodwind

# Part 1: Guitar String

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('Qr_rxqwc1jE')

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../introduction_to_digital_filters/')
from PlotUtils import plot_frequency_response
from ipython_animation import create_animation, DEFAULT_FPS
from DelayLine import DelayLine

from matplotlib import pyplot as plt
import numpy as np
from scipy.signal import freqz
from IPython.display import Audio

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (12, 4)
_ = np.seterr(divide='ignore')

In [ ]:
from DelayLine import DelayLine

def bandlimited_triangle_pluck(length):
    pluck = np.zeros(length)
    pluck[0:length // 4] = np.linspace(0, 1, length // 4)
    pluck[length // 4:length] = np.linspace(1, 0, length - length // 4)
    return np.fft.irfft(np.fft.rfft(pluck, length)[:length//3], length) # remove some HF components

class Waveguide:
    def __init__(self,
                 length_samples=21,
                 pickup_sample=4,
                 wave_variable_label='y'):
        self.length_samples = length_samples
        self.pickup_sample = pickup_sample
        self.d_l = DelayLine(length_samples)
        self.d_r = DelayLine(length_samples)
        self.set_initial_shape(bandlimited_triangle_pluck(length_samples))
        self.bridge_reflection_y_0 = 0 # memory for single pole lowpass filter at bridge
        self.wave_variable_label = wave_variable_label

    def set_initial_shape(self, y_init):
        y_l = y_init / 2
        y_r = y_init / 2
        self.d_r.tick_all(y_r[::-1])
        self.d_l.tick_all(y_l)

    def tick(self):
        r = self.d_r.get_tap(1) # least recent sample from rightgoing wave component - traveling into nut
        l = self.d_l.get_tap(1) # least recent sample from leftgoing wave component - traveling into bridge
        self.d_l.tick(-r) # negative reflection at right rigid termination (nut)
        self.d_r.tick(-l) # negative reflection at left termination (bridge)
        
        return self.tap_pickup()

    def tap_pickup(self, pickup_sample=None):
        pickup_sample = pickup_sample if pickup_sample is not None else self.pickup_sample
        return self.d_l.get_tap(pickup_sample) + self.d_r.get_tap(self.length_samples - pickup_sample)


In [ ]:
def animate_waveguide(waveguide, num_frames=None, fps=10, ticks_per_frame=1):
    L = waveguide.length_samples
    wv_label = waveguide.wave_variable_label
    L_indices = np.arange(L)
    d_l = waveguide.d_l
    d_r = waveguide.d_r
    num_frames = num_frames or (L - 1) * 2

    fig, (y_plot, y_r_plot, y_l_plot, pickup_plot) = plt.subplots(4, 1, figsize=(14, 4 * 3))

    y_r_ax, = y_r_plot.plot(d_r.get_all()[::-1], label='${}_r$'.format(wv_label),
                            linestyle='--', c='b', alpha=0.7, linewidth=2)
    y_l_ax, = y_l_plot.plot(d_l.get_all(), label='${}_l$'.format(wv_label),
                            linestyle='--', c='b', alpha=0.7, linewidth=2)
    y_ax, = y_plot.plot(d_r.get_all()[::-1] + d_l.get_all(), label='${0}_l + {0}_r$'.format(wv_label),
                        linewidth=3, c='b')

    y_plot.set_title('${0}(t,x) = {0}_r(t,x) + {0}_l(t,x)$'.format(wv_label), size=16)
    y_r_plot.set_title('${}_r(t,x)$'.format(wv_label), size=16)
    y_l_plot.set_title('${}_l(t,x)$'.format(wv_label), size=16)

    for plot in [y_r_plot, y_l_plot]:
        plot.axvline(waveguide.pickup_sample, label='Pickup $x$', linewidth=3, c='black')
    pickup_data = np.zeros(num_frames)
    pickup_data[0] = waveguide.tap_pickup(waveguide.pickup_sample)
    pickup_ax, = pickup_plot.plot(pickup_data, c='black')
    pickup_plot.set_xlim(0, num_frames - 1)
    pickup_plot.set_ylim(-1.2, 1.2)
    pickup_plot.grid(True)
    pickup_plot.set_title('Signal from pickup at $x={}$'.format(waveguide.pickup_sample), size=16)
    pickup_plot.set_xlabel('$t$ (samples) $\\longrightarrow$')

    for plot in [y_plot, y_r_plot, y_l_plot]:
        plot.legend()
        plot.grid(True)
        plot.set_ylabel('$y$')
        plot.set_ylim(-1.2, 1.2)
        plot.set_xlim(0, L - 1)
        plot.set_xticks(L_indices)

    y_plot.set_xlabel('$x$')
    for plot in [y_r_plot, y_l_plot]:
        plot.set_xlabel('Delay Line Sample Index')

    y_r_plot.set_xticklabels(L_indices[::-1])
    plt.tight_layout()

    def animate(frame):
        if frame < 1:
            return

        for _ in range(ticks_per_frame):
            waveguide.tick()

        y_r_x = np.roll(L_indices, -d_l.read_sample)[::-1]
        y_r = d_r.get_all()[y_r_x]
        y_r_ax.set_ydata(y_r)
        y_r_plot.set_xticklabels(y_r_x)

        y_l_x = np.roll(L_indices, -d_l.read_sample)
        y_l = d_l.get_all()[y_l_x]
        y_l_ax.set_ydata(y_l)
        y_l_plot.set_xticklabels(y_l_x)

        y_ax.set_ydata(y_r + y_l)

        pickup_data[frame] = waveguide.tap_pickup()
        pickup_ax.set_ydata(pickup_data)

    return create_animation(fig, plt, animate, frames=num_frames, frames_per_second=fps)

# Digital waveguide models

In [ ]:
waveguide = Waveguide()
animate_waveguide(waveguide, num_frames=waveguide.length_samples*4, fps=20, ticks_per_frame=2)

In [ ]:
sample_rate = fs = 44100
a_hz = 220
# Because of the finite sampling of the "string", with no interpolating delay line,
# this may not actually be 220Hz.
a_samples = int(fs / a_hz)

waveguide = Waveguide(length_samples=a_samples, pickup_sample=a_samples // 7)
waveguide_out = [waveguide.tick() for i in range(fs * 4)]
# get rid of harsh begin/end
fade_samples = fs//4
fade_ramp = np.linspace(0.0, 1.0, fade_samples)
waveguide_out[:fade_samples] *= fade_ramp
waveguide_out[-fade_samples:] *= fade_ramp[::-1]

### Audio

In [ ]:
Audio(waveguide_out, rate=fs)

The above animation shows the travelling-wave decomposition of string vibrations from an initial displacement. We're using displacement as the wave variable, but velocity and force waves move through the string in the same way. Displacement is chosen because:
* It's visually intuitive (looks like a real string)
* We can extract audio without additional integration
* We can easily implement collision detections

This first example uses a simple one-pole lowpass filter as a "yielding bridge" model (left side). We model the nut (right side) as a rigid termination that simply inverts the right-traveling wave. The energy that's lost at the yielding bridge is _transferred_ to the guitar body (or piano soundboard, etc.). That is, the energy that isn't reflected is transmitted. We don't model that here, but instead "tap" into the string directly by summing the two travelling components to get the physical displacement of the string at a specified point along its length.

# A yielding "bridge" termination

![](https://static.gibson.com/product-images/Acoustic/ACCSVU796/Antique%20Cherry/front-banner-1600_900.png)

In [ ]:
from Filters import OnePoleFilter

class WaveguideWithYieldingBridge(Waveguide):
    def __init__(self,
                 length_samples=21,
                 pickup_sample=4,
                 bridge_filter=None,
                 wave_variable_label='y'):
        super(WaveguideWithYieldingBridge, self).__init__(length_samples, pickup_sample, wave_variable_label)
        self.bridge_filter = bridge_filter or OnePoleFilter()

    def tick(self):
        r = self.d_r.get_tap(1) # least recent sample from rightgoing wave component - traveling into nut
        l = self.d_l.get_tap(1) # least recent sample from leftgoing wave component - traveling into bridge
        self.d_l.tick(-r) # negative reflection at right rigid termination (nut)
        self.d_r.tick(-self.bridge_filter.tick(l)) # negative, yielding reflection at left termination (bridge)

        return self.tap_pickup()

# A yielding "bridge" termination

In [ ]:
waveguide = WaveguideWithYieldingBridge()
animate_waveguide(waveguide, num_frames=waveguide.length_samples*8, fps=20, ticks_per_frame=2)

In [ ]:
waveguide = WaveguideWithYieldingBridge(length_samples=a_samples, pickup_sample=a_samples // 7)
waveguide_out = [waveguide.tick() for i in range(fs * 7)]

### Audio

In [ ]:
Audio(waveguide_out, rate=fs)

Notice that the low frequencies ring out for a very long time. This is because our simple bridge reflectance filter doesn't attenuate the lowest frequencies at all.

A small improvement in realism would be to slightly damp even the lowest frequencies:

In [ ]:
slightly_damped_bridge_filter = OnePoleFilter(0.498, 0.498)
waveguide = WaveguideWithYieldingBridge(length_samples=a_samples,
                                        pickup_sample=a_samples // 7,
                                        bridge_filter=slightly_damped_bridge_filter)
waveguide_out = [waveguide.tick() for i in range(fs * 6)]

In [ ]:
Audio(waveguide_out, rate=fs)

# String Excitation

Where we're headed here is modelling striking or plucking strings with things like hammers and picks (plectra). We'll look at affixed masses, struck strings and plucked strings. Along the way, we'll visualize and listen to the resulting sound with each addition. By the end, we'll end up with decently realistic-sounding models that are very computationally efficient.

In [ ]:
class PointMassReflectionFilter():
    def __init__(self, mass=1.0):
        # Treat string impedance * sampling interval as a constant for now.
        # We can vary the behavior we care about by only changing the mass.
        RT = 1.0
        denominator = (1 + RT / mass)
        denominator = max(denominator, 1e-10) # prevent divide-by-zero
        self.g = 1 / denominator
        self.p = (1 - RT / mass) / denominator
        self.p = min(self.p, 1 - 1e-8) # avoid pole right on the unit circle
        self.z_1 = 0.0
    
    def tick(self, in_sample):
        new_z_1 = self.g * in_sample + self.p * self.z_1
        out_sample = new_z_1 - self.z_1
        self.z_1 = new_z_1
        return out_sample

    def reset(self):
        self.z_1 = 0.0

In [ ]:
from enum import Enum 

# Explored later
class DampedSpringReflectionFilter():
    def __init__(self, k=1.0, damping=1.0):
        # Treat string impedance and sampling interval as constants for now.
        # We can vary the behavior we care about by only changing k and µ (damping)
        T = 1.0
        R = 1.0
        self.p = (1 - k * T / (2 * (damping + 2 * R))) / (1 + k * T / (2 * (damping + 2 * R)))
        self.zero = (1 - k * T / (2 * damping)) / (1 + k * T / (2 * damping))
        self.g = (1 - self.p) / (1 - self.zero)
        self.z_1 = 0.0
    
    def tick(self, in_sample):
        new_z_1 = self.g * in_sample + self.p * self.z_1
        out_sample = new_z_1 - self.zero * self.z_1
        self.z_1 = new_z_1
        return out_sample

    def reset(self):
        self.z_1 = 0.0

class WaveguideWithExcitation(Waveguide):
    ExcitationType = Enum('ExcitationType', 'affixed hammer plectrum pluck')

    def __init__(self,
                 length_samples=42,
                 pickup_sample=42//4,
                 excitation_sample=42//2,
                 tension=0.04,
                 excitation_mass=1.0,
                 spring_k=None,
                 spring_damping=None, # If both k & damping are provided, then a spring filter is used.
                 excitation_type=ExcitationType.hammer,
                 bridge_filter=None,
                 wave_variable_label='y'):
        self.length_samples = length_samples
        self.excitation_sample = excitation_sample
        self.pickup_sample = pickup_sample
        self.length_left = self.excitation_sample # number of samples to left of excitation
        self.length_right = self.length_samples - self.excitation_sample
        self.d_r1 = DelayLine(self.length_left)
        self.d_r2 = DelayLine(self.length_right)
        self.d_l1 = DelayLine(self.length_left)
        self.d_l2 = DelayLine(self.length_right)
        self.set_initial_shape(np.zeros(length_samples))
        if spring_k and spring_damping:
            self.excitation_reflection_filter = DampedSpringReflectionFilter(k=spring_k, damping=spring_damping)
        else:
            self.excitation_reflection_filter = PointMassReflectionFilter(mass=excitation_mass)
        # Bridge is a simple lowpass with slight damping at dc
        self.bridge_filter = bridge_filter or OnePoleFilter(0.498, 0.498)
        self.wave_variable_label = wave_variable_label
        # This "tension" is only used to determine how much the slope neighboring the
        # excitation "pushes back" against it.
        self.tension = tension
        self.excitation_type = excitation_type
        self.picking_up = True # always start with picking upwards (if excitation_type is plectrum)
        self.rest_excitation_y = 1.0
        self.excitation_velocity = 0.0 # y-units-per-sample
        self.deactivate_excitation(-self.rest_excitation_y)

    def set_initial_shape(self, y_init):
        y_r = y_init / 2
        y_l = y_init / 2

        self.d_r1.tick_all(y_r[:self.length_left][::-1])
        self.d_r2.tick_all(y_r[self.length_left:][::-1])
        self.d_l1.tick_all(y_l[:self.length_left])
        self.d_l2.tick_all(y_l[self.length_left:])

    # Set string shape back to 0s
    def reset(self):
        self.d_r1.clear()
        self.d_r2.clear()
        self.d_l1.clear()
        self.d_l2.clear()

    def get_all(self):
        y_r1 = np.roll(self.d_r1.get_all(), -self.d_r1.read_sample)[::-1]
        y_r2 = np.roll(self.d_r2.get_all(), -self.d_r2.read_sample)[::-1]
        y_l1 = np.roll(self.d_l1.get_all(), -self.d_l1.read_sample)
        y_l2 = np.roll(self.d_l2.get_all(), -self.d_l2.read_sample)
        left_y = y_r1 + y_l1
        right_y = y_r2 + y_l2
        into_excitation = self.into_excitation()
        excitation_y = self.excitation_y if into_excitation and not self.is_affixed() else (left_y[-1] + right_y[0]) / 2
        return np.concatenate([left_y, [excitation_y], right_y])

    # `velocity` is in units of y-units-per-sample.
    # It's scaled to a reasonable scale relative to the max desired string displacement (+/- 1)
    def trigger(self, velocity=1.0):
        velocity = velocity / 8
        self.excitation_reflection_filter.reset() # clear physical excitation state
        if self.is_plectrum() or self.is_hammer():
            if self.is_plectrum():
                self.picking_up = not self.picking_up
                if self.picking_up:
                    self.activate_excitation(-self.rest_excitation_y, velocity)
                else:
                    self.activate_excitation(self.rest_excitation_y, -velocity)
            elif self.is_hammer():
                self.activate_excitation(-self.rest_excitation_y, velocity)
        elif self.is_pluck() or self.is_affixed():
            self.set_initial_shape(bandlimited_triangle_pluck(self.length_samples))
        if self.is_affixed():
            self.tick_excitation()

    def tick(self, in_sample=0.0):
        into_excitation_rightgoing = self.d_r1.get_tap(1)
        into_right_termination = self.d_r2.get_tap(1)
        into_excitation_leftgoing = self.d_l2.get_tap(1)
        into_left_termination = self.d_l1.get_tap(1)

        if not self.is_affixed():
            self.tick_excitation()

        into_excitation = self.into_excitation()
        out_of_excitation = self.excitation_reflection_filter.tick(into_excitation) if into_excitation else 0

        self.d_r1.tick(-self.bridge_filter.tick(into_left_termination) + in_sample)
        self.d_l2.tick(-into_right_termination)
        self.d_r2.tick(into_excitation_rightgoing + out_of_excitation)
        self.d_l1.tick(into_excitation_leftgoing + out_of_excitation)

        if self.is_affixed():
            self.tick_excitation()
        return self.tap_pickup()

    def tick_excitation(self):
        if not self.is_excitation_active():
            return
        if self.is_affixed():
            self.excitation_y = (self.tap_pickup(self.excitation_sample - 1) + self.tap_pickup(self.excitation_sample)) / 2
        elif self.is_hammer() or self.is_plectrum():
            # force -> velocity
            opposing_force = self.force_opposing_excitation()
            if opposing_force > 0 and (self.is_hammer() or self.picking_up):
                self.excitation_velocity -= opposing_force
            elif opposing_force < 0 and self.is_plectrum() and not self.picking_up:
                self.excitation_velocity -= opposing_force

            # velocity -> displacement
            self.excitation_y += self.excitation_velocity # hammer/plectrum driving into the string

            # boundary conditions
            if self.excitation_y < -self.rest_excitation_y:
                self.deactivate_excitation(-self.rest_excitation_y)
            elif self.excitation_y > self.rest_excitation_y and self.is_plectrum():
                self.deactivate_excitation(self.rest_excitation_y)
            elif self.is_plectrum():
                if self.picking_up and self.excitation_velocity <= 0:
                    self.deactivate_excitation(self.rest_excitation_y)
                elif not self.picking_up and self.excitation_velocity >= 0:
                    self.deactivate_excitation(-self.rest_excitation_y)

    # How much is the string "pushing back" against the excitation mass?
    # (Proportional to slope of string to left and right of excitation)
    def force_opposing_excitation(self):
        slope_to_left = self.excitation_y - self.tap_pickup(self.excitation_sample - 1)
        slope_to_right = self.excitation_y - self.tap_pickup(self.excitation_sample)
        unscaled_force = (slope_to_left + slope_to_right) / 2
        return unscaled_force * self.tension

    def into_excitation(self):
        if not self.is_excitation_active():
            return None

        into_excitation_rightgoing = -self.d_r1.get_tap(1) # invert displacement wave variables
        into_excitation_leftgoing = -self.d_l2.get_tap(1)
        into_excitation = into_excitation_leftgoing + into_excitation_rightgoing
        if self.is_affixed():
            return into_excitation

        # Non-affixed excitations introduce energy into the string if they are actively colliding with it.
        into_excitation += self.excitation_y
        is_colliding = into_excitation > 0 and (self.is_hammer() or (self.is_plectrum() and self.picking_up))
        is_colliding = is_colliding or into_excitation < 0 and self.is_plectrum() and not self.picking_up
        # `None` signals to the caller that there is no collision and the excitation should be ignored.
        return into_excitation if is_colliding else None

    def tap_pickup(self, pickup_sample=None):
        pickup_sample = pickup_sample or self.pickup_sample
        pickup_sample %= self.length_samples

        if pickup_sample < self.length_left:
            return self.d_r1.get_tap(-pickup_sample - 1) + self.d_l1.get_tap(pickup_sample)
        else:
            pickup_sample -= self.length_left
            return self.d_r2.get_tap(-pickup_sample - 1) + self.d_l2.get_tap(pickup_sample)

    def is_excitation_active(self):
        return self.is_affixed() or self.excitation_active

    def activate_excitation(self, excitation_y=None, excitation_velocity=None):
        if excitation_y:
            self.excitation_y = excitation_y
        if excitation_velocity:
            self.excitation_velocity = excitation_velocity
        self.excitation_active = True

    def deactivate_excitation(self, excitation_y=None):
        self.excitation_active = False
        if excitation_y:
            self.excitation_y = excitation_y
        self.excitation_velocity = 0

    def is_affixed(self):
        return self.excitation_type == self.ExcitationType.affixed
    def is_plectrum(self):
        return self.excitation_type == self.ExcitationType.plectrum
    def is_hammer(self):
        return self.excitation_type == self.ExcitationType.hammer
    def is_pluck(self):
        return self.excitation_type == self.ExcitationType.pluck

In [ ]:
from matplotlib import gridspec

def animate_waveguide_with_excitation(waveguide,
                                      num_frames=None,
                                      fps=10,
                                      ticks_per_frame=1,
                                      on_frame={}):
    wv_label = waveguide.wave_variable_label
    indices_left = np.arange(waveguide.length_left)
    indices_right = np.arange(waveguide.length_right)
    d_r1 = waveguide.d_r1
    d_r2 = waveguide.d_r2
    d_l1 = waveguide.d_l1
    d_l2 = waveguide.d_l2

    num_frames = num_frames or (waveguide.length_samples - 1) * 2

    fig = plt.figure(constrained_layout=True, figsize=(14, 4 * 3))

    gs = gridspec.GridSpec(figure=fig, nrows=4, ncols=2,
                           width_ratios=[waveguide.length_left, waveguide.length_right])
    y_plot = fig.add_subplot(gs[0, :])
    y_r1_plot = fig.add_subplot(gs[1, 0])
    y_r2_plot = fig.add_subplot(gs[1, 1])
    y_l1_plot = fig.add_subplot(gs[2, 0])
    y_l2_plot = fig.add_subplot(gs[2, 1])
    pickup_plot = fig.add_subplot(gs[3, :])

    y_plot.axvline(waveguide.pickup_sample, label='Pickup $x$', linewidth=3, c='black')
    pickup_data = np.full(num_frames, np.nan)
    pickup_data[0] = waveguide.tap_pickup()
    pickup_ax, = pickup_plot.plot(pickup_data, c='black')
    pickup_plot.set_xlim(0, num_frames - 1)
    pickup_plot.set_ylim(-1.1, 1.1)
    pickup_plot.grid(True)
    pickup_plot.set_title('Signal from pickup at $x={}$'.format(waveguide.pickup_sample), size=16)
    pickup_plot.set_xlabel('$t$ (samples) $\\longrightarrow$')

    y_r1_ax, = y_r1_plot.plot(d_r1.get_all()[::-1], label='${}_1^+$'.format(wv_label),
                              linestyle='--', c='b', alpha=0.7, linewidth=2)
    y_r2_ax, = y_r2_plot.plot(d_r2.get_all()[::-1], label='${}_2^+$'.format(wv_label),
                              linestyle='--', c='b', alpha=0.7, linewidth=2)
    y_l1_ax, = y_l1_plot.plot(d_l1.get_all(), label='${}_1^-$'.format(wv_label),
                              linestyle='--', c='b', alpha=0.7, linewidth=2)
    y_l2_ax, = y_l2_plot.plot(d_l2.get_all(), label='${}_2^-$'.format(wv_label),
                              linestyle='--', c='b', alpha=0.7, linewidth=2)

    y_ax, = y_plot.plot(waveguide.get_all(), label='${0}^+ + {0}^-$'.format(wv_label), linewidth=3, c='b')

    excitation_y_ax = y_plot.scatter([waveguide.excitation_sample], [waveguide.excitation_y], c='red', label='Excitation', zorder=10)

    y_plot.set_title('${0}(t,x) = {0}^+(t,x) + {0}^-(t,x)$'.format(wv_label), size=16)
    y_r1_plot.set_title('${}_1^+(t,x) \\rightarrow$'.format(wv_label), size=16)
    y_r2_plot.set_title('${}_2^+(t,x) \\rightarrow$'.format(wv_label), size=16)
    y_l1_plot.set_title('${}_1^-(t,x) \\leftarrow$'.format(wv_label), size=16)
    y_l2_plot.set_title('${}_2^-(t,x) \\leftarrow$'.format(wv_label), size=16)

    for plot in [y_plot, y_r1_plot, y_l1_plot]:
        plot.set_ylabel('$y$')
    for plot in [y_r2_plot, y_l2_plot]:
        plot.set_yticklabels([])
        plot.set_ylabel('-')
    y_plot.set_xlim(0, waveguide.length_samples)
    y_plot.set_xlabel('$x$')
    for left_plot in [y_r1_plot, y_l1_plot]:
        left_plot.set_xlim(0, indices_left.size - 1)
        left_plot.set_xticks(indices_left)
    for right_plot in [y_r2_plot, y_l2_plot]:
        right_plot.set_xlim(0, indices_right.size - 1.03)
        right_plot.set_xticks(indices_right)
    
    y_l1_plot.axvline(0.03, label='Bridge Filter $y_n=gx_n + py_{n-1}$', linewidth=3, c='purple')
    y_r1_plot.axvline(indices_left.size - 1.03, label='Excitation Reflectance Filter $\\hat{\\rho}_{d}(z)$', linewidth=3, c='red')
    y_r2_plot.axvline(indices_right.size - 1.03, label='Nut Filter $y_n=-x_n$', linewidth=3, c='green')
    y_l2_plot.axvline(0.03, label='Excitation Reflectance Filter $\\hat{\\rho}_{d}(z)$', linewidth=3, c='red')

    for plot in [y_r1_plot, y_r2_plot, y_l1_plot, y_l2_plot]:
        plot.set_xlabel('Delay Line Sample Index')
    for plot in [y_plot, y_r1_plot, y_r2_plot, y_l1_plot, y_l2_plot]:
        plot.grid(True)
        plot.set_ylim(-1.2, 1.2)
        plot.legend(loc='upper right')

    y_r1_plot.set_xticklabels(indices_left[::-1])
    y_r2_plot.set_xticklabels(indices_right[::-1])

    def animate(frame):
        if frame < 1:
            waveguide.trigger()
            return

        action = on_frame.get(frame)
        if action:
            action()

        for _ in range(ticks_per_frame):
            waveguide.tick()

        y_r1_x = np.roll(indices_left, -d_r1.read_sample)[::-1]
        y_r1 = d_r1.get_all()[y_r1_x]
        y_r1_ax.set_ydata(y_r1)
        y_r1_plot.set_xticklabels(y_r1_x)

        y_r2_x = np.roll(indices_right, -d_r2.read_sample)[::-1]
        y_r2 = d_r2.get_all()[y_r2_x]
        y_r2_ax.set_ydata(y_r2)
        y_r2_plot.set_xticklabels(y_r2_x)

        y_l1_x = np.roll(indices_left, -d_l1.read_sample)
        y_l1 = d_l1.get_all()[y_l1_x]
        y_l1_ax.set_ydata(y_l1)
        y_l1_plot.set_xticklabels(y_l1_x)

        y_l2_x = np.roll(indices_right, -d_l2.read_sample)
        y_l2 = d_l2.get_all()[y_l2_x]
        y_l2_ax.set_ydata(y_l2)
        y_l2_plot.set_xticklabels(y_l2_x)

        y_ax.set_ydata(waveguide.get_all())
        excitation_y_ax.set_offsets([waveguide.excitation_sample, waveguide.excitation_y])
        excitation_y_ax.set_color('red' if waveguide.is_excitation_active() else 'gray')

        pickup_data[frame] = waveguide.tap_pickup()
        pickup_ax.set_ydata(pickup_data)

    return create_animation(fig, plt, animate, frames=num_frames, frames_per_second=fps)

## Affixed Mass

In [ ]:
animation_string_samples = 42
length_samples = animation_string_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples, 
                                    excitation_type=WaveguideWithExcitation.ExcitationType.affixed,
                                    excitation_sample=length_samples // 3,
                                    pickup_sample=length_samples // 5,
                                    excitation_mass=5.0)

waveguide.trigger()
animate_waveguide_with_excitation(waveguide, num_frames=waveguide.length_samples*4, fps=10, ticks_per_frame=1)

* We now have _four_ delay lines in our model - one for each traveling wave direction on each side of the attached mass.
* We now have _three_ filters in our model - the filters we've already seen at the "bridge" (left) and "nut" (right), as well as the new excitation reflectance filter
* We can imagine a single sample of string displacement traveling through the model like this:
  - Start at the top-left and travel to the right through the ($y_1^+$) delay line
  - When hitting the attached mass scattering junction, _reflect_ (and invert) part of my displacement into the right side of the bottom-left ($y_1^-$) delay line and start traveling through it to the left
  - The remainder of my displacement will be _trasmitted_ through the junction to the _right_ of the mass ($y_2^+$), and start traveling through it to the right
  - When I hit the right non-yielding termination ("nut"), I get reflected into the lower-right delay line ($y_2^-$) and start traveling through it to the left.
  - When hitting the attached mass scattering junction for the second time, reflect (and invert) part of my displacement into the left side of the top-right ($y_2^+$) delay line and start traveling through it to the right
  - The remainder of my displacement will be trasmitted into the delay line to the _left_ of the mass ($y_1^-$)
  - Finally, when I run into the "bridge" at the left end of $y_1^-$, reflect some of my displacement back into the top-left delay-line where I started. Some part of it will be transmitted into the instrument body via the bridge. (This is still the only lossy part of the model.)
* Since the mass in this example is _light_, notice how the lowest frequency energy is still allowed through, while higher frequency energy is reflected.
* No energy is _introduced_ into the string by the mass - what's not reflected from it is trasmitted through.

Let's listen to what this sounds like (the only thing changing is the delay line length):

In [ ]:
length_samples = a_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples, 
                                    excitation_type=WaveguideWithExcitation.ExcitationType.affixed,
                                    excitation_sample=length_samples // 3,
                                    pickup_sample=length_samples // 5,
                                    excitation_mass=5.0)

In [ ]:
waveguide.trigger()
Audio([waveguide.tick() for _ in range(fs * 6)], rate=fs)

We can clearly hear at least two disctinct tones. This is because we effectively have _three_ different string lengths interacting with each other - the string to the left of the mass, the string to the right, and the full string length.  The main tone still comes from the full string length, and the other buzzing tone comes from higher frequencies alternating between the left and right string sections. It's buzzy mostly because of a discontinuity introduced a the beginning, as the one-pole filter "warms up" its state (its history starts with a single zero value).

Now let's look at the behavior of a _heavy_ affixed mass:

## A heavier affixed mass

In [ ]:
animation_string_samples = 42
length_samples = animation_string_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples, 
                                    excitation_type=WaveguideWithExcitation.ExcitationType.affixed,
                                    excitation_sample=length_samples // 3,
                                    pickup_sample=length_samples // 5,
                                    excitation_mass=80.0)

waveguide.trigger()
animate_waveguide_with_excitation(waveguide, num_frames=waveguide.length_samples*4, fps=10, ticks_per_frame=1)

Notice how the _heavy_ mass prevents reflects all but just a little bit of low-frequency energy. It's not _quite_ heavy enough to act like a fully rigid termination, though - it still moves up and down a little bit.

Here's what it sounds like:

In [ ]:
length_samples = a_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples, 
                                    excitation_type=WaveguideWithExcitation.ExcitationType.affixed,
                                    excitation_sample=length_samples // 3,
                                    pickup_sample=length_samples // 5,
                                    excitation_mass=100.0)
waveguide.trigger()
Audio([waveguide.tick() for _ in range(fs * 6)], rate=fs)

This one sounds much more unusual! The pickup is placed to the left of the mass, so we hear a detuned sound composed mainly of the frequency relationship between the left half of the string and the entire string length. But we also get a buzzy higher-frequency tone that persists for longer than the rest of the sustain. Remember that the right-termination (the nut) doesn't attenuate _any_ frequencies. Since the heavy mass acts _almost_ as a rigid termination itself, the energy in the right half of the string is maintained for much longer. Only the small amount of low-frequency energy that "leaks through" the heavy affixed mass is able to be slowly dissippated through the yielding bridge. The high-frequency components are reflected almost completely and will be "stuck" in the right half of the string. It helps to again think of why this is the case, physically. High frequencies mean displacements are alternating rapidly between positive and negative values. The heavier the mass, the longer it takes for the mass to react to being "hit" by the energy of a large string displacement. Before it's able to react and move (yield), it gets pushed back in the opposite direction. The net result is that the mass effectively doesn't move at all in reaction to the high frequencies, and thus acts as a rigid termination for higher frequencies.

As we would expect, if we listen to what's happening to the _right_ of the mass, we'll be able to tap into those "trapped" high frequencies, and the low frequencies of the entire string will decay out at the normal rate:

In [ ]:
length_samples = a_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples, 
                                    excitation_type=WaveguideWithExcitation.ExcitationType.affixed,
                                    excitation_sample=length_samples // 3,
                                    pickup_sample=4 * length_samples // 5,
                                    excitation_mass=100.0)
waveguide.trigger()
Audio([waveguide.tick() for _ in range(fs * 6)], rate=fs)

Up to this point, the only way we've introduced energy into the string is by initializing the string position.

Most stringed instruments, however, are _excited_ by an external force like a guitar pick (plectrum) or a piano hammer.

First we consider the case of a _hammer_, which we will currently treat as our point mass colliding with the string from below. When the hammer is _in contact_ with the string, it displaces the string and introduces our mass scattering junction. In this specific model, I make the string "push back" on the hammer with a force proportional to the string slope directly surrounding the hammer point. I disengage the hammer (shown as gray) after it returns to a displacement of -1, in order to keep it in view but prevent any buzzing if the string rebounds back and hits it. (In a realtime instrument, all of these considerations could be provided as controls.)

# Incorporating Control Motion and Collision Detection

## Picking motion

Let's stick with our simple mass for a bit, and introduce a new control motion aspect - picking up and down.

In my model, the only change between a hammer and pick is that when the velocity of the pick starts to encounter enough resistance to _reverse direction_, it instead "slips off" of the string, and deactivates. Also, I store a little state to track whether we're picking up or down:

In [ ]:
length_samples = animation_string_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples,
                                    pickup_sample=length_samples//6,
                                    excitation_sample=length_samples//4,
                                    excitation_type=WaveguideWithExcitation.ExcitationType.plectrum,
                                    excitation_mass=8.0)
num_frames = waveguide.length_samples * 5
on_frame = {
    # trigger again halfway through to pick back down
    num_frames // 2: lambda: waveguide.trigger()
}

In [ ]:
animate_waveguide_with_excitation(waveguide, num_frames=num_frames, fps=20, ticks_per_frame=1, on_frame=on_frame)

In [ ]:
waveguide_out = np.zeros(fs * 10)

length_samples = a_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples,
                                    pickup_sample=length_samples//6,
                                    excitation_sample=length_samples//4,
                                    excitation_type=WaveguideWithExcitation.ExcitationType.plectrum,
                                    excitation_mass=5.0)

for i in range(waveguide_out.size):
    if i < fs * 5 and i % (fs // 6) == 0: # every 6th of a second starting at t=0, trigger again
        waveguide.trigger(np.random.rand())
    waveguide_out[i] = waveguide.tick()

In [ ]:
Audio(waveguide_out, rate=fs)

## A heavier mass

In [ ]:
waveguide_out = np.zeros(fs * 10)

length_samples = a_samples
waveguide = WaveguideWithExcitation(length_samples=length_samples,
                                    pickup_sample=length_samples//6,
                                    excitation_sample=length_samples//4,
                                    excitation_type=WaveguideWithExcitation.ExcitationType.plectrum,
                                    excitation_mass=200.0)

for i in range(waveguide_out.size):
    if i < fs * 5 and i % (fs // 6) == 0: # every 6th of a second starting at t=0, trigger again
        waveguide.trigger(np.random.rand())
    waveguide_out[i] = waveguide.tick()

In [ ]:
Audio(waveguide_out, rate=fs)

## Strum

In [ ]:
import sys
sys.path.append('../musimathics/')
from pitches import frequencies_for_note_labels
from Filters import IirFilter

class Guitar:
    def __init__(self, with_string_coupling=True):
        frequencies = frequencies_for_note_labels(['E3', 'A3', 'D4', 'G4', 'B4', 'E5'])
        sample_lengths = [round(fs / f) for f in frequencies] # TODO support fractional delay lengths
        self.strings = [WaveguideWithExcitation(length_samples=length_samples,
                                                pickup_sample=length_samples//6,
                                                excitation_sample=length_samples//4,
                                                excitation_type=WaveguideWithExcitation.ExcitationType.hammer,
                                                excitation_mass=10.0) for length_samples in sample_lengths]
        self.with_string_coupling = with_string_coupling
        if with_string_coupling:
            self.coupling_filter = IirFilter([0.1], [1.0, -0.9]) # one-pole
        self.coupling_gain = 0.007
        self.previous_out = 0.0

    def tick(self):
        out = 0.0
        for string in self.strings:
            if self.with_string_coupling:
                coupling_out = self.coupling_gain * self.coupling_filter.tick(self.previous_out)
                out += string.tick(coupling_out)
            else:
                out += string.tick()

        self.previous_out = out
        return out

    def trigger(self, velocity=1.0, string_index=None):
        string_index = string_index if string_index is not None else np.random.randint(len(self.strings))
        self.strings[string_index].trigger(velocity)
    
    # Convenience method to play all the string in short succession,
    # with gaps between triggers set by `trigger_delay`
    def strum(self, string_indices=np.arange(6), num_samples=fs*10, trigger_delay=fs//12):
        out = np.zeros(num_samples)
        for i in range(num_samples):
            if i % trigger_delay == 0 and i//trigger_delay < len(string_indices):
                string_index = string_indices[i//trigger_delay]
                self.trigger(np.random.uniform(low=0.5, high=1.0), string_index=string_index)
            out[i] = self.tick()
        return out

In [ ]:
guitar = Guitar()
string_indices = np.concatenate([np.arange(6), np.arange(6)[::-1]])
strum_samples = guitar.strum(string_indices=string_indices, num_samples=fs*10)
Audio(strum_samples, rate=fs)

## Impulse response for guitar body

In [ ]:
from scipy.io.wavfile import read as read_wav
from IPython.display import Audio

# From https://www.3sigmaaudio.com/items/category/classical-guitars/
fs, ir_samples = read_wav('classical_guitar_ir_16b.wav')
ir_samples = ir_samples / ir_samples.max() # normalize to [-1, 1]
Audio(ir_samples, rate=fs)

In [ ]:
def convolve(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    flipped_y = flip(y)
    convolved = np.zeros(x.size + y.size - 1)
    if x.dtype == complex or y.dtype == complex:
        convolved = convolved.astype(complex)
    for n in range(convolved.size):
        convolved[n] = np.sum(x * shift(flipped_y, n))
    return convolved

In [ ]:
def plot_with_clipped_edges(x, y, **args):
    return plt.plot(np.concatenate([[x[0]], x, [x[-1]]]), np.concatenate([[0], y, [0]]), **args)

def create_convolution_animation(f, g, normalize=False, title='', length_seconds=3.5):
    f = np.asarray(f)
    g = np.asarray(g)
    x_max = len(f) * 5
    num_frames = x_max + len(g) + 1

    f = f / f.max()
    fig = plt.figure(figsize=(10.5, 2))
    plt.title(title)
    plt.axhline(color='black', linewidth=0.5)
    f_start = (x_max - len(f)) // 2
    f_line, = plot_with_clipped_edges(np.arange(len(f)) + f_start, f, c='b', label='$x$')
    g_line, = plot_with_clipped_edges(np.arange(-len(g), 0), g, c='r', label='$y$')
    convolution = np.convolve(f, g)
    if normalize:
        convolution = convolution / np.abs(convolution).max()
    revealed_convolution = np.full(len(convolution), np.nan)
    conv_line, = plt.plot(np.arange(len(convolution)) + (x_max - len(convolution)) / 2, revealed_convolution, label='$y = conv(x, y)$', linewidth=3)
    plt.axis([0, x_max, min(0, convolution.min(), f.min(), g.min()), max(1, convolution.max(), f.max(), g.max()) + 0.1])
    plt.legend(loc='upper right')

    def animate(i):
        g_data = g_line.get_xdata() + 1
        g_line.set_xdata(g_data) # step convolution to the right
        g_pos = g_data[0]
        if g_pos + len(g) >= f_start and g_pos < f_start + len(f):
            n_revealed = g_pos + len(g) - f_start
            revealed_convolution[:n_revealed] = convolution[:n_revealed]
        conv_line.set_ydata(revealed_convolution)
    return create_animation(fig, plt, animate, length_seconds=length_seconds, frames_per_second=num_frames/length_seconds)

### Convolution

In [ ]:
x = [1.0] * 10 + [0.5] * 10 + [0] * 20
h = np.exp(-np.arange(40))
create_convolution_animation(x, h[::-1], normalize=True, title='Convolution: ADSR Envelope')

## Final Result

In [ ]:
from scipy.signal import convolve
strum_with_body = convolve(strum_samples, ir_samples)

Audio(strum_with_body, rate=fs)

# Part 2: Piano

## Commuted Synthesis

![](https://flylib.com/books/2/729/1/html/2/images/0131089897/graphics/01fig10.gif)

## Excitation Signals into Simple String Loop

In [ ]:
import numpy as np

import sys
sys.path.append('../')

from Filters import TwoZeroFilter, PoleZeroFilter, OnePoleFilter, IirFilter
sys.path.append('../musimathics')
from pitches import frequency_for_note_label, frequencies_for_note_labels
note2freq = frequency_for_note_label
notes2freqs = frequencies_for_note_labels

import matplotlib.pyplot as plt
import matplotlib.pylab as pl
%matplotlib inline
from scipy.signal import freqz
from IPython.display import Audio

from DelayLine import DelayLine
from SamplePlayer import SamplePlayer, WavPlayer
from AllpassDelay import AllpassDelay

fs = sample_rate = 44100
lowest_frequency = note2freq('A0')
highest_frequency = note2freq('C8')

In [ ]:
from scipy.signal import lfilter

soundboard_player = WavPlayer('soundboard.wav', normalize=True)

excite_size = soundboard_player.samples.size
noise = np.random.uniform(low=-1, high=1, size=excite_size)
decaying_noise = noise * np.exp(-np.linspace(0, 24, excite_size)) # found exp constant visually
filtered_decaying_noise = lfilter(b=[0.03], a=[1, -0.97], x=decaying_noise)
filtered_decaying_noise /= filtered_decaying_noise.max()

def plot_excitation_signals():
    fig, axes = plt.subplots(5, 1, figsize=(10, 12))
    t = np.arange(excite_size) / soundboard_player.fs
    axes[0].plot(t, noise)
    axes[0].grid(True)
    axes[0].set_title('Noise Signal')
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')

    axes[1].plot(t, decaying_noise)
    axes[1].grid(True)
    axes[1].set_title('Exponentially Decaying Noise')
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude')
    
    w, H = freqz([0.03], [1, -0.97])
    axes[2].plot(w, 20 * np.log10(np.abs(H)), c='red')
    axes[2].grid(True)
    axes[2].set_title('LP Filter')
    axes[2].set_xlabel('Frequency (radians/sample)')
    axes[2].set_ylabel('Magnitude (dB)')

    axes[3].plot(t, filtered_decaying_noise)
    axes[3].grid(True)
    axes[3].set_title('LP-Filtered Exponentially Decaying Noise')
    axes[3].set_xlabel('Time (s)')
    axes[3].set_ylabel('Normalized Amplitude')
    
    axes[4].plot(t, soundboard_player.samples)
    axes[4].grid(True)
    axes[4].set_title('Measured Piano Soundboard Impulse Response')
    axes[4].set_xlabel('Time (s)')
    axes[4].set_ylabel('Normalized amplitude')
    
    fig.tight_layout()

In [ ]:
plot_excitation_signals()

In [ ]:
out_samples = np.zeros(fs * 10) # keep reusing this to limit memory allocation and speed things up

In [ ]:
def string_loop(exciter=soundboard_player,
                frequency=frequency_for_note_label('A3'),
                loop_filter=OnePoleFilter(0.497, 0.497)): # simplest lowpass slightly damped at dc
    delay_length = sample_rate / frequency
    delay = AllpassDelay(delay_length, int(delay_length + 2))
    exciter.reset()
    for i in range(len(out_samples)):
        into_string = exciter.tick() + loop_filter.tick(delay.get_next_out())
        out_samples[i] = delay.tick(into_string)
    return out_samples

In [ ]:
Audio(noise, rate=soundboard_player.fs)

In [ ]:
Audio(string_loop(exciter=SamplePlayer(noise)), rate=soundboard_player.fs)

In [ ]:
Audio(decaying_noise, rate=soundboard_player.fs)

In [ ]:
Audio(string_loop(exciter=SamplePlayer(decaying_noise)), rate=soundboard_player.fs)

In [ ]:
Audio(filtered_decaying_noise, rate=soundboard_player.fs)

In [ ]:
Audio(string_loop(exciter=SamplePlayer(filtered_decaying_noise)), rate=soundboard_player.fs)

In [ ]:
Audio(soundboard_player.samples, rate=soundboard_player.fs)

In [ ]:
Audio(string_loop(exciter=soundboard_player), rate=soundboard_player.fs)

In [ ]:
Audio(string_loop(exciter=soundboard_player, frequency=note2freq('C5')), rate=soundboard_player.fs)

In [ ]:
soundboard_player_exp = SamplePlayer(soundboard_player.samples * np.exp(-np.linspace(0, 3, excite_size)))
Audio(string_loop(exciter=soundboard_player_exp, frequency=note2freq('C5')), rate=soundboard_player.fs)

# Loop Filter Design
## Energy Decay Relief (not using)

In [ ]:
def index_closest_to_value(array, value):
    return np.abs(array - value).argmin()

from scipy.signal import spectrogram
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import cm

def plot_edr(signal, fs=fs, nfft=1024, frequency_cutoff=6_000, harmonic_frequencies=None):
    # frame_size_ms = 30 # minimum frame length, in ms
    # overlap = 0.75 # fraction of frame overlapping
    # min_frame_length = int(fs * frame_size_ms / 1000)
    # Calculate fft size = next power of 2
    # frame_length = 2 ** int(np.ceil(np.log2(min_frame_length)))
    # F, T, B = spectrogram(signal, fs, 'hann', frame_length, overlap*frame_length)
    
    # default `spectrogram` overlap value, which will be used to calculate decay times later
    overlap_samples=256//8
    F, T, B = spectrogram(signal, fs, 'hann', noverlap=256//8, nfft=nfft)

    frequency_indices = np.arange(np.argmax(F > frequency_cutoff)) if harmonic_frequencies is None else [index_closest_to_value(F, frequency) for frequency in harmonic_frequencies]
    F = F[frequency_indices]
    B = B[frequency_indices,:]
    B_energy = B * np.conj(B)
    B_edr = np.fliplr(np.cumsum(np.fliplr(B_energy), axis=1))
    B_edr = 10 * np.log10(np.abs(B_edr)) # convert to dB
    B_edr -= B_edr.max() # normalize to 0 dB
    db_cutoff = -120
    B_edr = np.clip(B_edr, a_min=db_cutoff, a_max=None) # truncate below cutoff dB

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    if harmonic_frequencies is None:
        ax.plot_surface(*np.meshgrid(T, F/1000), B_edr, rstride=1, cstride=4, antialiased=False, cmap=cm.coolwarm)
        ax.set_ylabel('Frequency (kHz)')
    else:
        ax.plot_wireframe(*np.meshgrid(T, F/1000), B_edr, rstride=1, cstride=nfft // 20, antialiased=True, cmap=cm.coolwarm, linewidth=3)
        ax.set_yticks(F / 1000)
        ax.set_yticklabels(np.arange(F.size) + 1)
        ax.set_ylabel('Harmonic #')

    ax.view_init(30, 30) # orientation
    plt.title('Normalized Energy Decay Relief (EDR)', size=16)
    ax.set_xlabel('Time (s)'); _ = ax.set_zlabel('Magnitude (dB)')
    ax.set_zlim(db_cutoff + 2, 0) # + 2 hack to avoid small extra space matplotlib pads the z axis with
    return F, T, B_edr

In [ ]:
from scipy.io.wavfile import read as read_wav
fs_d4, d4 = read_wav('D4.wav')
Audio(d4, rate=fs_d4)

F, T, B_edr = plot_edr(d4, fs_d4)

In [ ]:
fundamental_frequency = note2freq('D4')
# higher `nfft` to get frequency bins with better alignment to exact harmonics
harmonic_frequencies = fundamental_frequency*np.arange(1, 13)
F, T, B_edr = plot_edr(d4, fs_d4, nfft=5096, harmonic_frequencies=harmonic_frequencies)

# Loop Filter Design
## Simple Filter with Brightness and Sustain Controls

### Length Three FIR Loop Filter with Brightness and Sustain

If we relax the DC normalization constraint so that two degrees of freedom are retained, we can then conveniently control the loop filter with a _brightness_ $B$ and a _sustain_ $S$ according to

$\begin{align}
g_0 &= e^{-6.91P/S}\\
g(0) &= g_0(1 + B) / 2\\
g(1) &= g_0(1 - B) / 4\\
\end{align}$,

where $P$ is the period in seconds (total loop delay), $S$ is the desired sustain time in seconds, the sustain parameter $S$ is the time to decay by -60 dB (or $\approx$ 6.91 time-constants) when brightness $B$ is maximum ($B = 1$) in which case the loop gain is $g_0$ at all frequencies.

In [ ]:
# Length-3 FIR filter with sustain and brightness controls
# From https://ccrma.stanford.edu/~jos/pasp/Length_FIR_Loop_Filter.html
def sustain_brightness_filter(sustain_seconds, brightness, frequency):
    g0 = np.exp(-6.91 / (sustain_seconds * frequency))
    b0 = g0 * (1 + brightness) / 2.0
    b1 = g0 * (1 - brightness) / 4.0
    loop_filter = TwoZeroFilter()
    loop_filter.set_coefficients(b1, b0, b1)

    return loop_filter

In [ ]:
def create_loop_filter_brightness_sustain_animation(animation_length_seconds=1):
    fig, frequency_plot = plt.subplots(1, 1, figsize=(10, 4))

    num_frames = DEFAULT_FPS * animation_length_seconds

    def animate(frame):
        brightness = 1 - frame / (num_frames - 1)
        sustain = np.exp(frame / 5)
        loop_delay_sec = 1 # assume loop delay of 1 sec for simplicity
        fig.suptitle('Length-3 FIR Loop Filter with Brightness ($B$) and Sustain ($S$) Controls\n$B={%0.2f}, S={%0.1f}$ sec' % (brightness, sustain), size=18)
        t60 = 6.91 # time-constants to decay to -60dB
        g = np.exp(-t60 * loop_delay_sec / sustain)
        g_0 = g * (1 + brightness) / 2
        g_1 = g * (1 - brightness) / 4
        b = [g_1, g_0, g_1]
        w, H = freqz(b, [1])

        frequency_plot.cla()
        plot_frequency_response(w, H, fig=fig, frequency_plot=frequency_plot, show_phase_plot=False, db=False)
        plt.subplots_adjust(top=0.70)    

    return create_animation(fig, plt, animate, length_seconds=animation_length_seconds, default_mode='reflect')

In [ ]:
create_loop_filter_brightness_sustain_animation(animation_length_seconds=1.5)

In [ ]:
notes_with_varying_sustain = [
    Audio(string_loop(loop_filter=sustain_brightness_filter(sustain, 0.6, note2freq('C5')),
                      exciter=soundboard_player_exp,
                      frequency=note2freq('A4')),
          rate=soundboard_player.fs)
    for sustain in [0.1, 1.0, 5.0]
]

### 0.1 seconds of sustain

In [ ]:
notes_with_varying_sustain[0]

### 1 second of sustain

In [ ]:
notes_with_varying_sustain[1]

### 5 seconds of sustain

In [ ]:
notes_with_varying_sustain[2]

In [ ]:
notes_with_varying_brightness = [
    Audio(string_loop(loop_filter=sustain_brightness_filter(3.0, brightness, note2freq('C5')),
                      exciter=soundboard_player_exp,
                      frequency=note2freq('A4')),
          rate=soundboard_player.fs)
    for brightness in np.linspace(0.01, 1.0, 3)
]

### Low brightness

In [ ]:
notes_with_varying_brightness[0]

### High brightness

In [ ]:
notes_with_varying_brightness[2]

# String Coupling

In [ ]:
# Initial t60s range from 15 seconds (A0) to 0.3 seconds (C8)
# Sustained t60s range from 50 seconds (A0) to 0.3 seconds (C8)
# From:
# Fletcher and Rossing "The Physics of Musical Instruments", 2nd edition
# Springer-Verlag, New York, 1998, p. 384
def initial_and_sustained_t60s(frequency,
                               max_initial_t60_sec=15.0, max_sustained_t60_sec=50.0):
    low_freq_log10_t60s = np.log10([max_initial_t60_sec, max_sustained_t60_sec])
    t60_scalar = (frequency - lowest_frequency) / (highest_frequency - lowest_frequency)
    t60s = 10.0 ** (low_freq_log10_t60s - t60_scalar * (low_freq_log10_t60s - np.log10(0.3)))
    t60_initial = t60s[0]
    t60_sustain = t60s[1]
    return t60_initial, t60_sustain

In [ ]:
def plot_initial_and_sustained_t60s(max_initial_t60_sec=15.0, max_sustained_t60_sec=50.0):
    frequencies = np.linspace(lowest_frequency, highest_frequency, 100)
    plt.figure(figsize=(16, 4))
    plt.plot(frequencies,
             [initial_and_sustained_t60s(frequency,
                                         max_initial_t60_sec=max_initial_t60_sec,
                                         max_sustained_t60_sec=max_sustained_t60_sec)
              for frequency in frequencies],
             linewidth=3, alpha=0.7)
    plt.legend(['Inital', 'Sustain'])
    plt.title('Initial and Sustained t60s by Frequency, Ranging from $A0$ to $C8$', size=16)
    plt.xlim(frequencies[0], frequencies[-1])
    plt.xticks(np.linspace(frequencies[0], frequencies[-1], 10))
    plt.xlabel('Frequency (Hz)')
    _ = plt.ylabel('t60 (s)')

In [ ]:
plot_initial_and_sustained_t60s(max_initial_t60_sec=9.0, max_sustained_t60_sec=20.0)

In [ ]:
# Simple utility to chain filters serially, used for dispersion filter which we'll cover later.
class FilterChain:
    def __init__(self, filters=[]):
        self.filters = filters
    
    def tick(self, in_sample):
        out_sample = in_sample
        for f in self.filters:
            out_sample = f.tick(out_sample)
        return out_sample
    
    def clear(self):
        for f in self.filters:
            f.clear()

In [ ]:
def string_loop_with_detuning(exciter=soundboard_player_exp,
                              frequency=note2freq('A4'),
                              frequency_jitter=0.002,
                              detuning=0.9993,
                              brightness=0.8,
                              strike_position=None, # No comb delay by default
                              max_initial_t60_sec=9.0,
                              max_sustained_t60_sec=20.0,
                              onset_sample=0,
                              gain=1.0,
                              # We'll get to dispersion filters later!
                              # This function should take a frequency and return a
                              # (filter, delay_compensation_samples) pair.
                              dispersion_filter_fn=None):

    if onset_sample == 0: # First note - reset the global samples array
        out_samples[:] = 0.0

    delay_length = sample_rate / frequency
    # slight detuning across the entire "string" for realism
    delay_length *= 1.0 + (np.random.uniform(low=-frequency_jitter, high=frequency_jitter) if frequency_jitter else 0)

    # minus 2 to compensate for extra filtering
    filter_delays = [-2,-2]
    dispersion_filters = None
    if dispersion_filter_fn is not None:
        dispersion_filters = [dispersion_filter_fn(frequency), dispersion_filter_fn(frequency * detuning)]
        for i in range(len(filter_delays)):
            filter_delays[i] += dispersion_filters[i][1]

    # Generalize to N strings
    delay_lines = [
        AllpassDelay(delay_length + filter_delays[0], int(delay_length)),
        AllpassDelay(delay_length * detuning + filter_delays[1], int(delay_length * detuning))
    ]

    t60_initial, t60_sustain = initial_and_sustained_t60s(frequency,
                                                          max_initial_t60_sec=max_initial_t60_sec,
                                                          max_sustained_t60_sec=max_sustained_t60_sec)
    loop_filters = [
        sustain_brightness_filter(t60_initial, brightness, frequency),
        sustain_brightness_filter(t60_sustain, brightness, frequency * detuning)
    ]

    if dispersion_filters is not None:
        for i in range(len(loop_filters)):
            loop_filters[i] = FilterChain([loop_filters[i], dispersion_filters[i][0]])

    if strike_position:
        strike_comb_delay = AllpassDelay(strike_position * delay_length, int(delay_length + 1))

    exciter.reset()

    delay_line_outs = [0.0, 0.0]

    for i in range(out_samples.size - onset_sample):
        in_sample = exciter.tick()
        gen_input = in_sample - (strike_comb_delay.tick(in_sample) if strike_position else 0.0)

        delay_line_outs[0] = loop_filters[0].tick(delay_lines[0].tick(gen_input + delay_line_outs[0]))
        # A hack to futher emulate a stronger attack.
        # Note that this technically invalidates the sustain t60 value.
        long_sustain_excite_gain = 0.6
        delay_line_outs[1] = loop_filters[1].tick(delay_lines[1].tick(long_sustain_excite_gain *
                                                                      gen_input + delay_line_outs[1]))

        out_samples[onset_sample + i] += sum(delay_line_outs) * gain

    return out_samples

In [ ]:
synthesized_d4 = string_loop_with_detuning(frequency=note2freq('D4'))
Audio(synthesized_d4, rate=soundboard_player.fs)

In [ ]:
from scipy.io.wavfile import read as read_wav
fs_d4, d4 = read_wav('D4.wav')
Audio(d4, rate=fs_d4)

In [ ]:
notes_with_varying_freq = [
    Audio(string_loop_with_detuning(frequency=frequency),
          rate=soundboard_player.fs)
    for frequency in notes2freqs(['A2', 'A4', 'A6'])
]

In [ ]:
notes_with_varying_freq[0]

In [ ]:
notes_with_varying_freq[1]

In [ ]:
notes_with_varying_freq[2]

# Strike Position

The position of the hammer strike along the length of the string is modelled as a simple comb filter. The physically-motivated idea is that strike position can be approximately modelled as a simple delay in one of the travelling wave components. (Think about the hammer sending energy down the string to the bridge. Since energy is transferred from the hammer into the string roughly symmetrically in both directions, there is another travelling componenet that bounces off the other string termination, the agraffe, with virtually no filtering, and later arrives at the bridge.)

First, we'll hear the effect of varying the "strike position" explicitly with the same frequency. Then we'll hear the effects of a simple method to calculate the strike position based on the frequency, based on measurements of a real piano.

In [ ]:
varying_strike_positions = [
    Audio(string_loop_with_detuning(strike_position=strike_position,
                                    frequency=note2freq('D4')),
          rate=soundboard_player.fs) for strike_position in np.linspace(1/20, 1/2, 2)
]

### Striking near end of string

In [ ]:
varying_strike_positions[0]

### Striking near middle of string

In [ ]:
varying_strike_positions[1]

# Strike Position
## Calculate from frequency - based on real piano measurements

In [ ]:
# Striking positions range from 0.122 (A0) to 0.115 (A4) to 0.08 (C8)
# Harold A. Conklin, Jr.
# "Design and tone in the mechanoacoustic piano. Part I. Piano hammers
# and tonal effects" Journal of the Acoustical Society of America
# Vol. 99, No. 6, June 1996, p. 3293
a4_frequency = 440.0
def strike_position(frequency):
    if frequency <= a4_frequency:
        strike_scalar = (np.log10(frequency) - np.log10(lowest_frequency)) / (np.log10(a4_frequency) - np.log10(lowest_frequency))
        return 0.122 - strike_scalar * (0.122 - 0.115)
    else:
        strike_scalar = (highest_frequency - frequency) / (highest_frequency - a4_frequency)
        return 0.08 + strike_scalar * (0.115 - 0.08)

In [ ]:
def plot_strike_position_by_frequency():
    frequencies = np.linspace(lowest_frequency, highest_frequency, 200)
    plt.figure(figsize=(16, 4))
    strike_positions = [strike_position(frequency) for frequency in frequencies]
    plt.plot(frequencies, strike_positions, linewidth=3, label='Strike Position')
    plt.hlines(0.115, xmin=lowest_frequency, xmax=highest_frequency, linestyle='--', alpha=0.5)
    plt.vlines([a4_frequency], ymin=0, ymax=1, label='$A4=0.115$')
    plt.title('Strike Position by Frequency, Ranging from $A0$ to $C8$', size=16)
    plt.xlim(frequencies[0], frequencies[-1])
    plt.ylim(min(strike_positions), max(strike_positions))
    plt.xticks(np.linspace(frequencies[0], frequencies[-1], 10))
    plt.xlabel('Frequency (Hz)')
    plt.legend()
    _ = plt.ylabel('Strike position (ratio of string length)')

In [ ]:
plot_strike_position_by_frequency()

In [ ]:
d4_frequency = note2freq('D4')
synthesized_d4 = string_loop_with_detuning(frequency=d4_frequency,
                                           strike_position=strike_position(d4_frequency))
Audio(synthesized_d4, rate=soundboard_player.fs)

# Modelling the hammer

Since the hammer acts as a nonlinear spring, the shape and time duration of the pulse varies with amplitude. A soft hit produces a wider pulse, whereas a strong hit produces a taller, narrow pulse. The duration of the pulse is also dependent on the hammer mass, so that lighter hammers used in the high registers have a shorter contact duration with the string and heavier hammers used in the lower registers have a longer contact time. The contact durations are approximately $0.5$ (ff) to $1.2$ (pp) ms at higher registers and $2.0$ (ff) to $4.0$ (pp) ms at lower registers. The frequency of the piano note determines the range of durations for the force pulse, and the amplitude value is used to linearly interpolate over this range.

In [ ]:
# Anders Askenfelt and Erik Janson "From touch to string vibrations"
# Five Lectures on the Acoustics of the Piano
# http://www.speech.kth.se/music/5_lectures/askenflt/askenflt.html
def hammer_pulse_and_gain(frequency, amplitude):
    # All in ms units:
    min_duration_hf = 0.5 # ff; min duration at high register
    min_duration_lf = 2.0 # ff; min duration at low register
    max_duration_hf = 1.2 # pp; max duration at high register
    max_duration_lf = 4.0 # pp; max duration at low register
    velocity_scalar = amplitude

    frequency_scalar = 0.0
    if frequency < lowest_frequency:
        frequency_scalar = 0.0
    elif frequency > highest_frequency:
        frequency_scalar = 1.0
    else:
        frequency_scalar = (np.log10(frequency) - np.log10(lowest_frequency)) / (np.log10(highest_frequency) - np.log10(lowest_frequency))

    min_duration = min_duration_lf - frequency_scalar * (min_duration_lf - min_duration_hf)
    max_duration = max_duration_lf - frequency_scalar * (max_duration_lf - max_duration_hf)
    pulse_duration = max_duration - velocity_scalar * (max_duration - min_duration)
    pulse_duration *= 0.5 # My addition - less LP filtering by shrinking every pulsee
    pulse_length = int(pulse_duration * sample_rate / 1000)
    # pulse_length = pulse_length // 2 TODO try this again
    pulse = 0.5 + 0.5 * np.cos(2 * np.pi * np.linspace(-0.5, 0.5, pulse_length))

    return pulse, amplitude / pulse.sum() # normalize so gain is 0 dB at DC

In [ ]:
def plot_hammer_pulses_and_gains(frequencies_and_amplitudes, title=''):
    colors = pl.cm.inferno(np.linspace(0,0.7,len(frequencies_and_amplitudes)))
    pulses_and_gains = [hammer_pulse_and_gain(frequency, amplitude)
                        for (frequency, amplitude) in frequencies_and_amplitudes]
    max_pulse_samples = max(pulse_and_gain[0].size for pulse_and_gain in pulses_and_gains)
    pulse_time_ms = np.linspace((-max_pulse_samples / sample_rate) / 2, (max_pulse_samples / sample_rate) / 2, max_pulse_samples) * 1000
    plt.figure(figsize=(10, 6))
    for i, pulse_and_gain in enumerate(pulses_and_gains):
        pulse = pulse_and_gain[0]
        gain = pulse_and_gain[1]
        centered_pulse = np.full(max_pulse_samples, np.nan)
        centered_pulse[:pulse.size] = pulse
        centered_pulse = np.roll(centered_pulse, (max_pulse_samples - pulse.size) // 2)
        plt.plot(pulse_time_ms, centered_pulse * gain, label='%0.2f Hz, Amp=%0.2f' %
                 frequencies_and_amplitudes[i], color=colors[i], linewidth=3)
    plt.title(title, size=16)
    plt.legend()
    plt.grid()
    plt.xlabel('Time (ms)')
    _ = plt.ylabel('Filter coefficient gain')

In [ ]:
frequencies_and_amplitudes = [(frequency_for_note_label(note_label), 0.9) for note_label in ['A1', 'C3', 'D4', 'G6']]
plot_hammer_pulses_and_gains(frequencies_and_amplitudes, title='Hammer Pulses for Varying String Frequency')

In [ ]:
frequencies_and_amplitudes = [(frequency_for_note_label('A1'), amplitude) for amplitude in np.linspace(0.1, 1, 5)]
plot_hammer_pulses_and_gains(frequencies_and_amplitudes, title='Hammer Pulses for Varying Strike Force')

In [ ]:
from scipy.signal import convolve

class HammerAndSoundboardExciter:
    def __init__(self, frequency=note2freq('A1'), amplitude=1.0):
        pulse, gain = hammer_pulse_and_gain(frequency, amplitude)
        # The STK implementation uses an all-zero filter for the pulse,
        # which can have a ton of coefficients (length of pulse).
        # Simply convolving them in the time-domain once, ahead of time, is _way_ faster.
        convolved = convolve(pulse * gain, soundboard_player_exp.samples)
        self.combined_player = SamplePlayer(convolved)
#         self.hammer_pulse = IirFilter()
#         self.hammer_pulse.set_b_coefficients(pulse)
#         self.hammer_pulse.set_gain(gain)
    
    def tick(self):
        return self.combined_player.tick()

    def reset(self):
        # TODO set new freq and amp
        self.combined_player.reset()

**Synthetic:**

In [ ]:
hammer_and_soundboard_exciter = HammerAndSoundboardExciter(frequency=d4_frequency)
synthetic_d4 = string_loop_with_detuning(exciter=hammer_and_soundboard_exciter,
                                         frequency=d4_frequency,
                                         brightness=0.8,
                                         strike_position=strike_position(d4_frequency))
Audio(synthetic_d4, rate=soundboard_player.fs)

**Real:**

In [ ]:
Audio(d4, rate=fs_d4)

# Dispersion Filters

In [ ]:
# Based on Faust implementation:
# https://github.com/grame-cncm/faustlibraries/blob/master/misceffects.lib#L193-L247
#
# "Dispersion Modeling in Waveguide Piano Synthesis Using Tunable
# Allpass Filters", by Jukka Rauhala and Vesa Valimaki, DAFX-2006, pp. 71-76
# <http://www.dafx.ca/proceedings/papers/p_071.pdf>
# is corrected in Dr. Rauhala's encompassing dissertation (and below).)
# <http://www.acoustics.hut.fi/research/asp/piano/>
def piano_dispersion_filter(f0, M=8, B=0.0001):
    # This is a black-box to me. See links above.
    wT = 2*np.pi*f0/fs
    Bc = max(B, 0.000001)
    k1 = -0.00179; k2 = -0.0233; k3 = -2.93
    kd = np.exp(k1*np.log(Bc)*np.log(Bc) + k2*np.log(Bc) + k3)
    m1 = 0.0126; m2 = 0.0606; m3 = -0.00825; m4 = 1.97
    Cd = np.exp((m1*np.log(M)+m2)*np.log(Bc)+m3*np.log(M)+m4)
    trt = 2**(1.0/12.0)
    Ikey = np.log(f0*trt/27.5)/np.log(trt)
    D = np.exp(Cd - Ikey*kd)
    a1 = (1 - D)/(1 + D) # D >= 0, so a1 >= 0

    polydel = lambda a: np.arctan(np.sin(wT)/(a + np.cos(wT)))/wT
    Df0 = polydel(a1) - polydel(1.0/a1)

    return FilterChain([PoleZeroFilter(a1, 1, a1) for i in range(M)]), -Df0*M

In [ ]:
from PlotUtils import plot_frequency_response
from ipython_animation import create_animation, DEFAULT_FPS

def create_dispersion_filter_animation(animation_length_seconds=1.5, fps=DEFAULT_FPS):
    fig, [frequency_plot, phase_plot] = plt.subplots(2, 1, figsize=(10, 6))

    num_frames = fps * animation_length_seconds

    min_frequency, max_frequency = note2freq('C1'), note2freq('A4')
    impulse = np.concatenate([[1.0], np.zeros(511)])

    def animate(frame):
        frequency = min_frequency + (max_frequency - min_frequency) * frame / (num_frames - 1)
        dispersion_filter, max_delay_samples = piano_dispersion_filter(frequency)

        h = [dispersion_filter.tick(sample) for sample in impulse]
        H = np.fft.rfft(h)
        w = np.linspace(0, np.pi, H.size)
        frequency_plot.cla()
        phase_plot.cla()
        plot_frequency_response(w, H, fig,
                                frequency_plot, phase_plot,
                                lower_db_lim=-20, phase_delay=True)
        frequency_plot.set_title('Magnitude Response of Dispersion Filter (Allpass)', size=16)
        phase_plot.set_title('Phase Response of Dispersion Filter, $f={%0.1f} Hz$' % frequency, size=16)
        phase_plot.hlines([max_delay_samples], xmin=w[0], xmax=w[-1], label='Max delay samples\n(for compensation)', linestyle='--')
        phase_plot.set_ylim(-160, np.pi)
        plt.tight_layout()

        _ = phase_plot.legend(loc='lower right')

    return create_animation(fig, plt, animate, length_seconds=animation_length_seconds, default_mode='reflect')

In [ ]:
create_dispersion_filter_animation()

In [ ]:
class PianoString:
    def __init__(self,
                 brightness=0.9,
                 frequency_jitter=0.001,
                 detuning=0.9993,
                 gain=1.0,
                 # We'll get to dispersion filters later!
                 # This function should take a frequency and return a
                 # (filter, delay_compensation_samples) pair.
                 dispersion_filter_fn=None,
                 model_hammer=False,
                 model_strike_position=False):
        self.frequency = None
        self.brightness = brightness
        self.frequency_jitter = frequency_jitter
        self.detuning = detuning
        self.strike_position = strike_position
        self.gain = gain
        self.loop_gain_target = self.loop_gain = 1.0
        self.dispersion_filter_fn = dispersion_filter_fn
        delay_length = fs / lowest_frequency
        self.delay_lines = [
            AllpassDelay(delay_length, int(delay_length) + 1),
            AllpassDelay(delay_length * detuning , int(delay_length * detuning) + 1)
        ]
        self.strike_comb_delay = AllpassDelay(delay_length, int(delay_length) + 1)

        self.model_hammer = model_hammer
        self.model_strike_position = model_strike_position
        self.delay_line_outs = [0.0, 0.0]

    def set_frequency(self, frequency):
        if self.frequency == frequency:
            return
        self.frequency = frequency
        delay_length = fs / frequency
        # slight detuning across the entire "string" for realism
        delay_length *= 1.0 + (np.random.uniform(low=-self.frequency_jitter, high=self.frequency_jitter) if self.frequency_jitter else 0)
        
        # minus 2 to compensate for extra filtering
        filter_delays = [-2,-2]
        self.dispersion_filters = None
        if self.dispersion_filter_fn is not None:
            self.dispersion_filters = [self.dispersion_filter_fn(frequency),
                                       self.dispersion_filter_fn(frequency * self.detuning)]
            for i in range(len(self.dispersion_filters)):
                filter_delays[i] += self.dispersion_filters[i][1]
                self.dispersion_filters[i] = self.dispersion_filters[i][0]

        self.delay_lines[0].set_delay_samples(delay_length + filter_delays[0])
        self.delay_lines[1].set_delay_samples(delay_length * self.detuning + filter_delays[1])

        self.t60_initial, self.t60_sustain = initial_and_sustained_t60s(frequency,
                                                                        max_initial_t60_sec=9.0,
                                                                        max_sustained_t60_sec=20.0)
        self.strike_comb_delay.set_delay_samples(strike_position(frequency) * delay_length)
        self.update_loop_filters()

    def set_brightness(self, brightness):
        self.brightness = brightness
        self.update_loop_filters()

    def update_loop_filters(self):
        self.loop_filters = [
            sustain_brightness_filter(self.t60_initial, self.brightness, self.frequency),
            sustain_brightness_filter(self.t60_sustain, self.brightness, self.frequency * self.detuning)
        ]
        
    def tick(self):
        if self.loop_gain < self.loop_gain_target:
            self.loop_gain += 0.0001
        elif self.loop_gain > self.loop_gain_target:
            self.loop_gain -= 0.0001

        in_sample = self.exciter.tick()
        gen_input = in_sample
        if self.model_strike_position:
            gen_input -= self.strike_comb_delay.tick(in_sample)

        # A hack to futher emulate a stronger attack.
        # Note that this technically invalidates the sustain t60 value.
        long_sustain_excite_gain = 0.6
        gen_inputs = [gen_input, long_sustain_excite_gain * gen_input]

        for i in range(len(self.delay_lines)):
            into_loop_filter = self.delay_lines[i].tick(gen_inputs[i] + self.delay_line_outs[i])
            back_into_delay_line = self.loop_filters[i].tick(into_loop_filter)
            if self.dispersion_filters is not None:
                back_into_delay_line = self.dispersion_filters[i].tick(back_into_delay_line)
            self.delay_line_outs[i] = self.loop_gain * back_into_delay_line

        return self.gain * sum(self.delay_line_outs)

    def note_on(self, frequency, amplitude):
        self.set_frequency(frequency)
        # Re-instantiating the exciter for every new note for simplicity.
        self.exciter = HammerAndSoundboardExciter(frequency, amplitude) if self.model_hammer else SamplePlayer(soundboard_player_exp.samples)
        self.exciter.reset()
        self.loop_gain_target = 1.0

    # off-amplitude is not currently used
    def note_off(self, amplitude=1.0):
        # Damping coefficients range from 0.75 (A0) to 0.9 (A5 and above)
        # Julien Bensa, "Analyse et Synthese de sons de piano par modeles
        # physiques et de signaux", These de doctorat de l'universite de la
        # Mediterranee Aix-Marseille II, 2003, p. 140
        a5_frequency = note2freq('A5')
        if self.frequency <= a5_frequency:
            loop_scalar = (self.frequency - lowest_frequency) / (a5_frequency - lowest_frequency)
            self.loop_gain_target = 0.75 + loop_scalar * (0.9 - 0.75)
        else:
            self.loop_gain_target = 0.9

In [ ]:
# create a bank of strings to reuse over all examples
piano_strings = [PianoString() for _ in range(10)]

In [ ]:
def arpeggiate(frequencies, onset_samples,
               brightness=0.9, model_hammer=True, model_strike_position=True,
               dispersion_filter_fn=None):

    assert(len(frequencies) == len(onset_samples))
    assert(len(frequencies) <= len(piano_strings))
    out_samples[:] = 0.0
    # "play" lower notes more quietly for a more even sound
    amplitudes = np.interp(frequencies, [min(frequencies), max(frequencies)], [1.0, 1.0])
    # I found that very small amplitudes in the hammer model are just way too filtered
    # (see flat, wide pulse curves in above charts).
    # At the same time, mixing them lower sounds more realistic to me.
    gains = np.interp(frequencies, [min(frequencies), max(frequencies)], [0.75, 1.0])

    for i in range(len(frequencies)):
        piano_strings[i].dispersion_filter_fn = dispersion_filter_fn
        piano_strings[i].model_hammer = model_hammer
        piano_strings[i].model_strike_position = model_strike_position
        piano_strings[i].brightness = brightness
        piano_strings[i].gain = gains[i]
        piano_strings[i].note_on(frequencies[i], amplitudes[i])

    out_samples[:] = 0.0
    for string_i, onset_sample in enumerate(onset_samples):
        for sample_i in range(out_samples.size - onset_sample):
            out_samples[onset_sample + sample_i] += piano_strings[string_i].tick()
#             if sample_i == off_sample:
#                 piano_strings[i].note_off(1.0)
    return out_samples

In [ ]:
# three A major triads
note_frequencies = frequencies_for_note_labels(['A2', 'C#3', 'E3',
                                                'A4', 'C#5', 'E5',
                                                'A5', 'C#6', 'E6'])
onset_samples = np.logspace(np.log10(fs / 2), np.log10(fs * 4), len(note_frequencies)).astype('int')
onset_samples -= onset_samples[0] # trigger first at sample 0

### With dispersion

In [ ]:
arpeggiate(note_frequencies, onset_samples,
           brightness=0.8,
           model_hammer=False,
           model_strike_position=False,
           dispersion_filter_fn=piano_dispersion_filter)
Audio(out_samples, rate=fs)

### Without dispersion

In [ ]:
arpeggiate(note_frequencies, onset_samples,
           brightness=0.8,
           model_hammer=False,
           model_strike_position=False,
           dispersion_filter_fn=None)
Audio(out_samples, rate=fs)

### Modeling strike position & dispersion

In [ ]:
arpeggiate(note_frequencies, onset_samples,
           brightness=0.7,
           model_hammer=False,
           model_strike_position=True,
           dispersion_filter_fn=piano_dispersion_filter)
Audio(out_samples, rate=fs)

### Modeling strike position, hammer pulse and dispersion

In [ ]:
arpeggiate(note_frequencies, onset_samples,
           brightness=0.98,
           model_hammer=True,
           model_strike_position=True,
           dispersion_filter_fn=piano_dispersion_filter)
Audio(out_samples, rate=fs)

In [ ]:
piano_string = PianoString(brightness=0.98,
                           model_hammer=True,
                           model_strike_position=True,
                           dispersion_filter_fn=piano_dispersion_filter)

num_out_samples = fs * 10
piano_striking_single_key = np.zeros(num_out_samples)
# Hit key every quarter second and release after a random short duration
note_on_samples = np.arange(0, fs * 6, fs // 4)
note_off_samples = note_on_samples + np.random.randint(low=fs//30, high=fs//6, size=note_on_samples.size)
# "hold down" last note until a second before the end
note_off_samples[-1] = num_out_samples - fs

for i in range(num_out_samples):
    if i in note_on_samples:
        amp = np.random.uniform(low=0.7, high=1.0) if i != note_on_samples[-1] else 1.0 # end with a loud note
        piano_string.note_on(note2freq('A#1'), amp)
    elif i in note_off_samples:
        piano_string.note_off()

    piano_striking_single_key[i] = piano_string.tick()

In [ ]:
Audio(piano_striking_single_key, rate=fs)

# Part 3: Woodwinds

In [ ]:
import numpy as np

import sys
sys.path.append('../')

from Filters import OneZeroFilter
from Generators import NoiseGenerator, SineGenerator
from AllpassDelay import AllpassDelay

sample_rate = fs = 44100
lowest_frequency = note2freq('A0')
highest_frequency = note2freq('C8')

In [ ]:
# This class implements a simple one breakpoint,
# non-linear reed function, as described by
# Smith (1986).  This function is based on a
# memoryless non-linear spring model of the reed
# (the reed mass is ignored) which saturates when
# the reed collides with the mouthpiece facing.

# See McIntyre, Schumacher, & Woodhouse (1983),
# Smith (1986), Hirschman, Cook, Scavone, and
# others for more information.
class ReedTable:
    def __init__(self):
        self.offset = 0.6
        self.slope = -0.8

    def tick(self, in_sample):
        # The input is differential pressure across the reed.
        out_sample = self.offset + self.slope * in_sample

        # If output is > 1, the reed has slammed shut and the
        # reflection function value saturates at 1.0.
        out_sample = min(out_sample, 1.0)

        # This is nearly impossible in a physical system, but
        # a reflection function value of -1.0 corresponds to
        # an open end (and no discontinuity in bore profile).
        out_sample = max(-1.0, out_sample)

        return out_sample

    def key_on(self):
        self.set_target(1.0)

    def key_off(self):
        self.set_target(0.0)

    def set_offset(self, offset):
        self.offset = offset
        
    def set_slope(self, slope):
        self.slope = slope

class EnvelopeGenerator:
    def __init__(self):
        self.target = 0.0
        self.value = 0.0
        self.rate = 0.001
        self.state = False

    def tick(self):
        if self.state:
            if self.target > self.value:
                self.value += self.rate
                if self.value >= self.target:
                    self.value = self.target
                    self.state = 0
            else:
                self.value -= self.rate
                if self.value <= self.target:
                    self.value = self.target
                    self.state = False
        return self.value

    def set_rate(self, rate):
        self.rate = rate
    
    def set_target(self, target):
        self.target = target
        if self.value != self.target:
            self.state = True

# envelope + noise + vibrato
class BreathPressureGenerator:
    def __init__(self):
        self.envelope = EnvelopeGenerator()
        self.noise = NoiseGenerator(gain=0.2)
        self.vibrato = SineGenerator()
        self.vibrato.set_frequency(5.735)
        self.vibrato.amplitude = 0.05
        
        self.last_envelope_out = 0.0
        self.last_noise_out = 0.0
        self.last_vibrato_out = 0.0

    def tick(self):
        self.last_noise_out = self.noise.tick()
        self.last_vibrato_out = self.vibrato.tick()

        self.last_envelope_out = self.envelope.tick()
        breath_pressure = self.last_envelope_out
        breath_pressure += breath_pressure * self.last_noise_out
        breath_pressure += breath_pressure * self.last_vibrato_out

        return breath_pressure

    # Apply breath pressure to instrument with given amplitude and rate of increase.
    def start_blowing(self, amplitude, rate):
        assert(amplitude > 0 and rate > 0)

        self.envelope.set_rate(rate)
        self.envelope.set_target(amplitude)

    # Decrease breath pressure with given rate of decrease.
    def stop_blowing(self, rate):
        self.envelope.set_rate(rate)
        self.envelope.set_target(0.0)

    def set_vibrato_frequency(self, vibrato_frequency):
        self.vibrato.set_frequency(vibrato_frequency)

    def set_vibrato_gain(self, vibrato_gain):
        self.vibrato.amplitude = vibrato_gain
    
    def set_noise_gain(self, noise_gain):
        self.noise.gain = noise_gain

    
class Clarinet:
    def __init__(self):
        self.filter = OneZeroFilter()
        self.breath_pressure_gen = BreathPressureGenerator()
        self.output_gain = 1.0

        max_delay_samples = 0.5 * fs / lowest_frequency
        self.delay_line = AllpassDelay(max_delay_samples, int(max_delay_samples) + 1)

        self.reed_table = ReedTable()
        self.set_reed_stiffness(0.5)

        self.set_frequency(note2freq('A3'))
        self.clear()

    def tick(self):
        self.last_breath_pressure_out = self.breath_pressure_gen.tick()
        # Perform commuted loss filtering.
        self.last_pressure_diff = -0.95 * self.filter.tick(self.delay_line.last_out)
        # Calculate pressure difference of reflected and mouthpiece pressures.
        self.last_pressure_diff -= self.last_breath_pressure_out
        self.last_reed_table_out = self.reed_table.tick(self.last_pressure_diff)
        # Perform non-linear scattering using pressure difference in reed function.
        return self.output_gain * self.delay_line.tick(self.last_breath_pressure_out +
                                                       self.last_pressure_diff*self.last_reed_table_out)

    def note_on(self, frequency, amplitude):
        self.set_frequency(frequency)
        self.breath_pressure_gen.start_blowing(0.55 + (amplitude * 0.3), amplitude * 0.001)
        self.output_gain = amplitude + 0.001

    def note_off(self, amplitude):
        self.breath_pressure_gen.stop_blowing(amplitude * 0.01)

    def set_frequency(self, frequency):
        # Account for filter delay and one sample "last out" delay.
        delay = 0.5 * (fs / frequency) - 1.0#self.filter.phase_delay(frequency) - 1.0
        self.delay_line.set_delay_samples(delay)

    # Realtime controls
    def set_reed_stiffness(self, reed_stiffness):
        self.reed_table.set_slope(-0.44 + (0.26 * reed_stiffness))
    def set_noise_gain(self, noise_gain):
        self.breath_pressure_gen.set_noise_gain(noise_gain)
    def set_vibrato_frequency(self, vibrato_frequency):
        self.breath_pressure_gen.set_vibrato_frequency(vibrato_frequency)
    def set_vibrato_gain(self, vibrato_gain):
        self.breath_pressure_gen.set_vibrato_gain(vibrato_gain)

    def clear(self):
        self.delay_line.clear()
        self.filter.clear()

In [ ]:
t = np.arange(4 * fs) / fs
out_samples = np.zeros(t.size)

def plot_woodwind_components():
    fig, plots = plt.subplots(7, 1, figsize=(12, 16))
    [sine_plot, noise_plot, envelope_plot, breath_plot, pressure_diff_plot, reed_table_plot, clarinet_plot] = plots
    
    clarinet = Clarinet()
    clarinet.set_reed_stiffness(0.57)
    
    triangle = np.concatenate([np.linspace(0, 1, t.size // 2),
                               np.linspace(1, 0, t.size // 2)])
    noise_gain_ramp = triangle * 0.15
    vibrato_frequency_ramp = triangle * 5
    vibrato_gain_ramp = triangle * 0.02
    
    sine_y = np.zeros(t.size)
    noise_y = np.zeros(t.size)
    envelope_y = np.zeros(t.size)
    breath_y = np.zeros(t.size)
    pressure_diff_y = np.zeros(t.size)
    reed_table_out_samples = np.zeros(t.size)
    
    breath_duration_samples = int(4*t.size//5) # hold for 4/5 of the duration
    clarinet.note_on(note2freq('F4'), 0.7)
    
    for i in range(t.size):
        if i == breath_duration_samples:
            clarinet.note_off(0.005)
    
        clarinet.set_vibrato_frequency(vibrato_frequency_ramp[i])
        clarinet.set_vibrato_gain(vibrato_gain_ramp[i])
        clarinet.set_noise_gain(noise_gain_ramp[i])
    
        out_samples[i] = clarinet.tick()
    
        sine_y[i] = clarinet.breath_pressure_gen.last_vibrato_out
        noise_y[i] = clarinet.breath_pressure_gen.last_noise_out
        envelope_y[i] = clarinet.breath_pressure_gen.last_envelope_out
        breath_y[i] = clarinet.last_breath_pressure_out
        pressure_diff_y[i] = clarinet.last_pressure_diff
        reed_table_out_samples[i] = clarinet.last_reed_table_out
    
    sine_plot.plot(t, sine_y)
    sine_plot.set_title('Vibrato with Modulated Frequency and Amplitude')
    
    noise_plot.plot(t, noise_y)
    noise_plot.set_title('Noise Generator with Modulated Gain')
    
    envelope_plot.plot(t, envelope_y)
    envelope_plot.set_title('Breath Pressure Envelope Generator')
    
    breath_plot.plot(t, breath_y)
    breath_plot.set_title('Breath Pressure Generator with Modulated Vibrato and Noise Gain')
    
    pressure_diff_plot.plot(t, pressure_diff_y)
    pressure_diff_plot.set_title('Pressure Diff (Reed Table Input)')
    
    reed_table_plot.plot(t, reed_table_out_samples)
    reed_table_plot.set_title('Reed Table Output Samples')
    
    clarinet_plot.plot(t, out_samples)
    clarinet_plot.set_title('Clarinet Samples')
    
    for plot in plots:
        plot.set_xlim(0, t.size / fs)
        plot.set_xlabel('$t$ (sec)')
        plot.set_ylabel('out sample')
    
    plt.tight_layout()

In [ ]:
plot_woodwind_components()

In [ ]:
Audio(out_samples, rate=fs)

In [ ]:
clarinet = Clarinet()
clarinet.set_reed_stiffness(0.65)
clarinet.set_noise_gain(0.3)
clarinet.set_vibrato_gain(0.025)

out_samples = np.zeros(fs * 6)
note_frequencies = notes2freqs(['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5'])
note_on_samples = np.arange(len(note_frequencies)) * fs // 2
note_off_samples = note_on_samples + 4 * fs // 10
note_off_samples[-1] = out_samples.size - fs # hold last note til a second before the end

note_index = 0
for i in range(out_samples.size):
    if np.isin(i, note_on_samples):
        amplitude = np.random.uniform(low=0.7, high=1.0)
        clarinet.note_on(note_frequencies[note_index], amplitude)
        note_index += 1
    elif np.isin(i, note_off_samples):
        clarinet.note_off(0.008)
    out_samples[i] = clarinet.tick()

Audio(out_samples, rate=fs)